In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

# Modelo de Regressão

In [2]:
# Carregar os dados - Regressão
df = pd.read_csv("datasets/colesterol.csv")

In [3]:
# Visualizar estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [4]:
df.describe()

,Id,Idade,Peso,Altura,Colesterol
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,49.160000,76.487500,174.868000,199.453260
std,288.819436,17.494755,21.411086,8.855337,50.625152
min,1.000000,20.000000,15.400000,160.000000,36.260000
25%,250.750000,34.000000,61.775000,167.000000,166.345000
50%,500.500000,48.500000,75.450000,175.000000,199.455000
75%,750.250000,64.250000,89.975000,183.000000,233.785000
max,1000.000000,79.000000,158.100000,190.000000,372.670000


In [5]:
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33,85.1,186,199.63
1,2,A,Não,Moderado,68,105.0,184,236.98
2,3,O,Não,Alto,25,64.8,180,161.79
3,4,A,Não,Alto,43,120.2,167,336.24
4,5,AB,Não,Baixo,79,88.5,175,226.23
5,6,B,Não,Baixo,68,66.8,170,185.31
6,7,A,Sim,Baixo,60,117.3,181,289.33
7,8,O,Sim,Moderado,35,86.9,174,216.48
8,9,O,Não,Baixo,62,81.3,166,235.30
9,10,B,Sim,Alto,44,32.7,165,97.79


In [6]:
# Ajustar Dataframe
df.drop(columns=["Id"], axis=1, inplace=True)

# Aplicar One Hot Encoding nas variaveis categóricas
df = pd.get_dummies(df, columns=["Grupo Sanguíneo", "Fumante", "Nível de Atividade"])

In [7]:
df.head(10)

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,0,0,1,0,0,1,0,1,0
1,68,105.0,184,236.98,1,0,0,0,1,0,0,0,1
2,25,64.8,180,161.79,0,0,0,1,1,0,1,0,0
3,43,120.2,167,336.24,1,0,0,0,1,0,1,0,0
4,79,88.5,175,226.23,0,1,0,0,1,0,0,1,0
5,68,66.8,170,185.31,0,0,1,0,1,0,0,1,0
6,60,117.3,181,289.33,1,0,0,0,0,1,0,1,0
7,35,86.9,174,216.48,0,0,0,1,0,1,0,0,1
8,62,81.3,166,235.30,0,0,0,1,1,0,0,1,0
9,44,32.7,165,97.79,0,0,1,0,0,1,1,0,0


### Treinar modelo de regressão linear multipla com RFE

In [8]:
# Separar X e y
X = df.drop(columns=["Colesterol"])
y = df["Colesterol"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [10]:
# Treinar o modelo com RFE
# RFE (Rercursive Feature Elimination)
# Uso um estimador e defino uma quantidade de features (hiperparâmetros)
# O RFE faz iterações inciciando com todas as features e eliminando a cada iteração até atinger a quantidade definida
# Elimina as características/features menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [11]:
# Quais features foram selecionadas
X_train.columns[rfe_method.get_support()]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto'],
      dtype='object')

In [12]:
# Ranking de Features
def mostrar_ranking(metodo_fs, X_train):
    # Obter o ranking das features
    ranking = metodo_fs.ranking_
    # Obter nome das features
    nomes_features = X_train.columns.to_list()
    # Criar DataFrame com o ranking + nome das features
    df_ranking = pd.DataFrame({"Feature": nomes_features, "Ranking": ranking})
    df_ranking = df_ranking.sort_values(by="Ranking", ascending=True)
    # exibir o ranking
    print(df_ranking)

In [13]:
# Ranking de Features do RFE Regressão
mostrar_ranking(rfe_method, X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
4            Grupo Sanguíneo_AB        1
7                   Fumante_Não        1
8                   Fumante_Sim        1
9       Nível de Atividade_Alto        1
5             Grupo Sanguíneo_B        2
3             Grupo Sanguíneo_A        3
11  Nível de Atividade_Moderado        4
10     Nível de Atividade_Baixo        5
6             Grupo Sanguíneo_O        6
0                         Idade        7


In [14]:
# Função para avaliar performance
def performance_regressao(modelo, X_test, y_test):
    # Faz predição com o modelo no conjunto de teste
    y_pred = modelo.predict(X_test)
    # Avaliar desempenho
    mse = mean_squared_error(y_test, y_pred, squared=False)
    return mse

In [15]:
performance_regressao(rfe_method, X_test, y_test)

/home/vitor-linux/.local/share/virtualenvs/Topicos_Complementares-PXZMFool/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.948793762634496

### Treinar modelo sem RFE

In [16]:
# Treinar modelo sem RFE
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [17]:
# Performance regressão sem RFE
performance_regressao(model_reg, X_test, y_test)

/home/vitor-linux/.local/share/virtualenvs/Topicos_Complementares-PXZMFool/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.017674671581466

### Treinar Modelo de Regressão Linear com RFECV

In [18]:
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [19]:
performance_regressao(rfe_method_cv, X_test, y_test)

/home/vitor-linux/.local/share/virtualenvs/Topicos_Complementares-PXZMFool/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.02288233120335

In [20]:
# Quais features foram selecionadas
X_train.columns[rfe_method_cv.get_support()]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [21]:
# Quantas features foram selecionadas
rfe_method_cv.n_features_

10

### Treinar modelo de regressão com SelectFromModel

In [33]:
sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.5)

In [34]:
# Quais features foram selecionadas
X_train.columns[sfm_method.get_support()]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Nível de Atividade_Alto'], dtype='object')

In [35]:
# Treinar modelo com as features selecionadas
X_train_ajustado_reg = sfm_method.transform(X_train)
X_test_ajustado_reg = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado_reg, y_train)

LinearRegression()

In [36]:
performance_regressao(model_reg, X_test_ajustado_reg, y_test)

/home/vitor-linux/.local/share/virtualenvs/Topicos_Complementares-PXZMFool/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.024674426990273

# Modelo de Classificação

### Carregar dados

In [45]:
# Carregar os dados - Regressão
df2 = pd.read_csv("datasets/fruit_quality.csv")

In [46]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4000 non-null   float64
 8   Quality      4000 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 281.4+ KB


In [47]:
df2.head(10)

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good
5,5,-3.425400,-1.409082,-1.913511,-0.555775,-3.853071,1.914616,-2.981523,bad
6,6,1.331606,1.635956,0.875974,-1.677798,3.106344,-1.847417,2.414171,good
7,7,-1.995462,-0.428958,1.530644,-0.742972,0.158834,0.974438,-1.470125,good
8,8,-3.867632,-3.734514,0.986429,-1.207655,2.292873,4.080921,-4.871905,bad
9,9,-0.727983,-0.442820,-4.092223,0.597513,0.393714,1.620857,2.185608,bad


In [48]:
# Ajustar Dataframe
df2.drop(columns=["A_id"], axis=1, inplace=True)
df2["Quality"] = (df2["Quality"] == 'good').astype(int)
df2

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1
...,...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,0
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,1
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,0
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,1


### Treinar modelo  de regressão logistica com RFE

In [49]:
# Separar X e y
X = df2.drop(columns=["Quality"])
y = df2["Quality"]

In [50]:
# Separar Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [80]:
rfe_method = RFE(estimator=LogisticRegression(), n_features_to_select=5)
rfe_method.fit(X_train, y_train)

RFE(estimator=LogisticRegression(), n_features_to_select=5)

In [81]:
# Quais features foram selecionadas
X_train.columns[rfe_method.get_support()]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [82]:
mostrar_ranking(rfe_method, X_train)

       Feature  Ranking
0         Size        1
1       Weight        1
2    Sweetness        1
4    Juiciness        1
6      Acidity        1
5     Ripeness        2
3  Crunchiness        3


In [83]:
def performance_classificacao(modelo, X_test, y_test):
    # Faz predição com o modelo no conjunto de teste
    y_pred = modelo.predict(X_test)
    # Avaliar desempenho
    return f1_score(y_test, y_pred)



In [84]:
performance_classificacao(rfe_method, X_test, y_test)

0.7738193869096934

### Treinar modelo sem RFE

In [85]:
# Treinar sem RFE
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression()

In [86]:
# Validar performance
performance_classificacao(model_lr, X_test, y_test)

0.7787903893951947